# Apply color grading, enhance contrast and add scale bars to epifluorescence images

In [ ]:
# Tools to read in the image files and filenames
import glob
import os
import re 

# Calculation and data frame tools
import numpy as np
import pandas as pd

# Image processing tools
import skimage
import skimage.io
import skimage.morphology

___

This code will create a new folders in the previous directory called X_LUT containing tif files of the images with adjusted contrast (using the skimage.equalize_adapthist function), LUT and scale bar. It will also merge two fluorescence channels of the same image and save them in a new folder called X_LUT_merged. 

Name files as follows: "conditions_merge-identifier_magnification_channel.tif" (for example: "X_Y_Z_area1_10x_RFP.tif" and "X_Y_Z_area1_10x_GFP.tif")

Written by Laura Luebbert, 22nd of March 2020.

___

# Load in the data:

Load images into Python:

In [ ]:
# Define the directory containing daytime data
data_dir = ''

# Glob string for images (loads all .tif files)
im_glob = os.path.join(data_dir, '*.tif')

# Get list of files in directory
im_list = sorted(glob.glob(im_glob))

# Let's look at the first 10 entries
im_list[:10]

# Define the parameters:

In [ ]:
# Define the interpixel distance in µm.
interpixel_distance_10x = 2
interpixel_distance_20x = 1

# Define the desired scale bar size in µm.
scale_bar = 100

# Define green, blue or red LUT for each channel
blue_channel = "DAPI"
green_channel = "GFP"
red_channel = "RFP"

___

# Take a look at some information using the first image:

Load the first tif file using scikit-image to get some information about bit-size and color channels:

In [2]:
# Read in the first tif file using skimage.
# In im_list[x], x defines which image is displayed.
im = skimage.io.imread(im_list[0])

im.shape
# Let's get information about the image.
print(
    "The image is stored as a",
    type(im),
    "of",
    im.dtype,
    "bits.",
    "The image consists of",
    im.shape[0],
    "x",
    im.shape[1],
    "pixels.",
)

___

# Enhance contrast and apply LUT according to channel:

In [3]:
# Empty array to save images with enhanced contract and LUT to
max_ims = []

for i, file in enumerate(im_list):
    # Read in each tif file using skimage
    max_image = skimage.io.imread(im_list[i])

    # local contrast enhancement based on histograms computed over different tile regions of the image
    max_image = skimage.exposure.equalize_adapthist(max_image)

    ## Add color
    if im_list[i].split("_")[-1].split(".")[0] == blue_channel:
        merged_im = []

        # Append red channel
        # Since we don't have a red channel here, I'll append an array of zeros (Skimage needs all three channels to save rgb images.)
        merged_im.append(np.zeros((max_image.shape)))

        # Append green channel
        merged_im.append(np.zeros((max_image.shape)))

        # Append blue channel
        merged_im.append(max_image)

        # Reorder dimensions of array to fit image representation in numpy
        merged_im = np.moveaxis(merged_im, 0, 2)
        merged_im = np.array(merged_im)

        max_ims.append(merged_im)

    if im_list[i].split("_")[-1].split(".")[0] == green_channel:
        merged_im = []

        # Append red channel
        merged_im.append(np.zeros((max_image.shape)))

        # Append green channel
        merged_im.append(max_image)

        # Append blue channel
        merged_im.append(np.zeros((max_image.shape)))

        # Reorder dimensions of array to fit image representation in numpy
        merged_im = np.moveaxis(merged_im, 0, 2)
        merged_im = np.array(merged_im)

        max_ims.append(merged_im)

    if im_list[i].split("_")[-1].split(".")[0] == red_channel:
        merged_im = []

        # Append red channel
        merged_im.append(max_image)

        # Append green channel
        merged_im.append(np.zeros((max_image.shape)))

        # Append blue channel
        merged_im.append(np.zeros((max_image.shape)))

        merged_im = np.array(merged_im)

        # Reorder dimensions of array to fit image representation in numpy
        merged_im = np.moveaxis(merged_im, 0, 2)
        merged_im = np.array(merged_im)

        max_ims.append(merged_im)

# Save the images as an 8-bit tif file:

### Convert images to 8-bit:

In [4]:
# Scale down images to 8 bits to save them as a jpeg using skimage.
max_ims_8 = []

for i, file in enumerate(max_ims):
    # Linearly scale down to 8-bit.
    image = (max_ims[i] / max_ims[i].max()) * 255

    # Change list to array and change type to 8-bit array.
    image = np.array(image)
    image = image.astype(np.uint8)

    max_ims_8.append(image)

# Change entire array "ims" to 8-bit array.
max_ims_8 = np.array(max_ims_8)
max_ims_8 = max_ims_8.astype(np.uint8)

### Burn in scale bars:

In [5]:
max_ims_8_with_scalebars = []

scalebar_10 = 1 / interpixel_distance_10x * scale_bar
scalebar_20 = 1 / interpixel_distance_20x * scale_bar

for i, file in enumerate(max_ims_8):
    # Make a copy of the image.
    image_with_scalebar = max_ims_8[i]

    if im_list[i].split("_")[-2] == "10x":
        # Burn the scale bar by changing pixel values (white scalebar = 1000 (for black set to = 0)).
        image_with_scalebar[970:980, 1190 : 1190 + int(scalebar_10)] = 1000

        # Append to new array.
        max_ims_8_with_scalebars.append(image_with_scalebar)

    if im_list[i].split("_")[-2] == "20x":
        # Burn the scale bar by changing pixel values (white scalebar = 1000 (for black set to = 0)).
        image_with_scalebar[970:980, 1100 : 1100 + int(scalebar_20)] = 1000

        # Append to new array.
        max_ims_8_with_scalebars.append(image_with_scalebar)

# Display one image with the scalebar.
skimage.io.imshow(max_ims_8_with_scalebars[0])

### Create folder to save images to:

In [6]:
path = ("{}/{}_LUT").format(
    "/".join(im_list[0].split("/")[:-2]), im_list[0].split("/")[-3]
)

os.mkdir(path)

### Slice out image names:

In [7]:
# Save the filename of the image in array as a first step to get the image title.
files = []

for filename in im_list:
    files.append(filename.split("/")[-1])

# Save image titles in array.
imnames = []

for name in files:
    imnames.append(name.split(".")[0])

### Save images:

In [8]:
# Save all images with the scale bar.
for i, image in enumerate(max_ims_8_with_scalebars):
    skimage.io.imsave(
        ("{}/{}_LUT/{}_LUT.tif").format(
            "/".join(im_list[0].split("/")[:-2]), im_list[0].split("/")[-3], imnames[i]
        ),
        max_ims_8_with_scalebars[i],
        plugin=None,
        check_contrast=False,
    )

___

# Merge different channels of the same image

In [9]:
# Define regex to find number in file name.
regex = re.compile(r"\d+")

# Array of files already visited.
files_done = []

# Define array to append image names to.
merged_imnames = []

# Define array to append merged images to.
merged_max_ims = []

for i, file in enumerate(im_list):
    # Condition that prevents the same file to be visited twice.
    if im_list[i] not in files_done:

        # Slice out third to last piece of filename following "_" of one file.
        lastname = im_list[i].split("_")[-3]

        # Find number in "lastname".
        number1 = regex.findall(lastname)

        files_done.append(im_list[i])

        for j, file in enumerate(im_list):
            if im_list[j] not in files_done:
                # Slice out third to last piece of filename following "_" of all other files for comparison.
                lastname = im_list[j].split("_")[-3]

                number2 = regex.findall(lastname)

                # Compute max projection and merge images if:
                # The sample number is the same,
                # and the entire filename is not the same,
                # and the channel is not the same,
                # and the condition is the same.
                if (
                    (number1 == number2)
                    and (im_list[i] != im_list[j])
                    and (im_list[i].split("_")[-1] != im_list[j].split("_")[-1])
                    and (
                        "_".join(im_list[i].split("_")[:-3])
                        == "_".join(im_list[i].split("_")[:-3])
                    )
                ):

                    # Save the new image title for the merged images in an array.
                    first = im_list[i].split("/")[-1]
                    second = "_".join(first.split("_")[:-1])
                    third = im_list[i].split(".")[-1]
                    name = [second, third]
                    merged_imnames.append(".".join(name))

                    # Read in image 1.
                    max_image1 = skimage.io.imread(im_list[i])

                    # Read in image 2.
                    max_image2 = skimage.io.imread(im_list[j])

                    ## Merge images 1 and 2.
                    # Empty array to save merged image to
                    merged_im = []

                    # Append red channel.
                    merged_im.append(max_image2)

                    # Append green channel.
                    merged_im.append(max_image1)

                    # Append blue channel.
                    # Since we don't have a blue channel here, I'll append an array of zeros.
                    merged_im.append(np.zeros((max_image1.shape)))

                    merged_im = np.array(merged_im)

                    # Reorder dimensions of array to fit image representation in numpy.
                    merged_im = np.moveaxis(merged_im, 0, 2)
                    merged_im = np.array(merged_im)

                    merged_max_ims.append(merged_im)

                    break

# Save the merged images as an 8-bit tif file:

### Convert images to 8-bit:

In [10]:
# Scale down images to 8 bits to save them using skimage.
merged_max_ims_8 = []

for i, file in enumerate(merged_max_ims):
    # Linearly scale down to 8-bit.
    image = (merged_max_ims[i] / merged_max_ims[i].max()) * 255

    # Change list to array and change type to 8-bit array.
    image = np.array(image)
    image = image.astype(np.uint8)

    merged_max_ims_8.append(image)

# Change entire array "ims" to 8-bit array.
merged_max_ims_8 = np.array(merged_max_ims_8)
merged_max_ims_8 = merged_max_ims_8.astype(np.uint8)

### Burn in scale bars:

In [11]:
merged_max_ims_8_with_scalebars = []

scalebar_10 = 1 / interpixel_distance_10x * scale_bar
scalebar_20 = 1 / interpixel_distance_20x * scale_bar

for i, file in enumerate(merged_max_ims_8):
    # Make a copy of the image.
    image_with_scalebar = merged_max_ims_8[i]

    if merged_imnames[i].split("_")[-1].split(".")[0] == "10x":
        # Burn the scale bar by changing pixel values (white scalebar = 1000 (for black set to = 0)).
        image_with_scalebar[970:980, 1190 : 1190 + int(scalebar_10)] = 1000

        # Append to new array.
        merged_max_ims_8_with_scalebars.append(image_with_scalebar)

    if merged_imnames[i].split("_")[-1].split(".")[0] == "20x":
        # Burn the scale bar by changing pixel values (white scalebar = 1000 (for black set to = 0)).
        image_with_scalebar[970:980, 1100 : 1100 + int(scalebar_20)] = 1000

        # Append to new array.
        merged_max_ims_8_with_scalebars.append(image_with_scalebar)

# Display one image with the scalebar.
skimage.io.imshow(merged_max_ims_8_with_scalebars[1])

### Create folder to save images to:

In [12]:
path = ("{}/{}_LUT_merged").format(
    "/".join(im_list[0].split("/")[:-2]), im_list[0].split("/")[-3]
)

os.mkdir(path)

### Save images:

In [13]:
# Save all images with the scale bar.
for i, image in enumerate(merged_max_ims_8_with_scalebars):
    skimage.io.imsave(
        ("{}/{}_LUT_merged/{}").format(
            "/".join(im_list[0].split("/")[:-2]),
            im_list[0].split("/")[-3],
            merged_imnames[i],
        ),
        merged_max_ims_8_with_scalebars[i],
        plugin=None,
        check_contrast=False,
    )

___

# Computing environment

In [ ]:
%load_ext watermark
%watermark -v -p re,numpy,pandas,skimage,jupyterlab